In [ ]:
import tweepy
from tkinter import *
import tkinter
from time import sleep
from datetime import datetime
from textblob import TextBlob
import matplotlib.pyplot as plt

In [ ]:
#Authentication

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
import csv
l ='@RahulGandhi'
tweets1 = []
backoff_counter = 0
try:
    for status in tweepy.Cursor(api.user_timeline,screen_name=l,tweet_mode =  "extended").items(100):
        tweets1.append(status)
except tweepy.TweepError as e:
    print(e.reason)
    sleep(60*backoff_counter)
    backoff_counter += 1

In [ ]:
tweet_id = []
for tweet in tweets1:
    tweet_id.append(tweet.id)

In [ ]:
print(max(tweet_id)-min(tweet_id))

In [ ]:
with open('tweets.csv','a') as f:
    writer = csv.writer(f)
    a = [tweets1]
    writer.writerows(a)

In [ ]:
def plotfunc(tweet,replies):
    polarity_list = []
    numbers_list = []
    number = 0
    for i in replies:
        try:
            print('analysing for sentiment analysis')
            analysis = TextBlob(i.full_text)
            analysis = analysis.sentiment
            polarity = analysis.polarity
            polarity_list.append(polarity)
            numbers_list.append(number)
            number = number + 1
        except tweepy.TweepError as e:
            print(e.reason)
        except StopIteration:
            break
    
#Plotting
    axes = plt.gca()
    axes.set_ylim([-1, 2])

    plt.scatter(numbers_list, polarity_list)

    averagePolarity = (sum(polarity_list))/(len(polarity_list))
    averagePolarity = "{0:.0f}%".format(averagePolarity * 100)
    time  = datetime.now().strftime("At: %H:%M\nOn: %m-%d-%y")

    plt.text(0, 1.25, "Average Sentiment:  " + str(averagePolarity) + "\n" + time, fontsize=12, bbox = dict(facecolor='none', edgecolor='black', boxstyle='square, pad = 1'))

    plt.title("Sentiment of"+tweet.text+"on Twitter")
    plt.xlabel("Number of Tweets")
    plt.ylabel("Sentiment")
    plt.show()

In [ ]:
replies=[] 
non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

for full_tweets in tweepy.Cursor(api.user_timeline,screen_name='RahulGandhi',timeout=999999,max_id = 1089854868175638528).items(100):
    for tweet in tweepy.Cursor(api.search,q='to:RahulGandhi', since_id=full_tweets.id, result_type='recent',timeout=999999).items(1000):
        if hasattr(tweet, 'in_reply_to_status_id_str'):
            if (tweet.in_reply_to_status_id_str==full_tweets.id_str):
                t = [tweet.text,tweet.author.screen_name,tweet.created_at]
                with open('comments4.csv','a') as f:
                    writer= csv.writer(f)
                    row = [full_tweets.id,tweet.text]
                    writer.writerows([row])
                    replies.append(tweet.text)
    print("Tweet :",full_tweets.text.translate(non_bmp_map))
    for elements in replies:
        print("Replies :",elements)
    replies.clear()

In [ ]:
import pandas as pd
from textblob  import TextBlob
df = pd.read_csv('comments4.csv')
df.size